# Implementação 1: Toy Example

Esse notebook apresenta a primeira implementação do Toy Example proposto.
Sua proposta segue os seguintes objetivos:
* A implementação não seguirá padrões sugeridos nos campos das dimensões da IAR.
* Esse notebook servirá de checagem para análise da eficiência e corretude da checklist proposta pelo NIAR.

## Processamento dos dados

Os dados são de origem do Datasus, contendo informações das internações nos períodos de 2022 à novembro de 2025 (dados mais recentes quando feito esse Toy Example). Devido ao tamanho dos dados brutos, não se encontram na página do Github. Para a execução desse notebook, não é necessário baixá-los, uma vez que o csv final, usado com os dados já processados, se encontra no diretório "SIH_Dados", dentro do próprio Github.

O download dos dados foi feito por conexão ftp com o servidor disponibilizado pelo datasus: ftp.datasus.gov.br. Especificamente, os dados se encontram no diretório "/dissemin/publicos/SIHSUS/200801_/Dados/". Foram usados todos os arquivos que começam com "RD" e contém "22", "23", "24", "25", no quinto e sexto caractére (quando escrito esse notebook, exclui-se os dados de dezembro de 2025 e os dados de novembro de 2025 dos estados "AC" e "RR", pois não foram divulgados ainda).

A fim de evitar refazer grandes cálculos toda vez que esse notebook é executado, o processamento dos dados - filtragem de colunas de interesse e cálculo de novas colunas - foi feito previamente. No diretório SIH_Dados, encontram-se os scripts usados para cada etapa do processamento:
* dbc_dbf.py e dbf_csv.py: Transformam os arquivos de dados do formato dbc para dbf e dbf para csv, respectivamente.
* filtrar.py: Lê os arquivos csv, filtram as colunas (e o "target" de interesse, DIAG_PRIN = "J...") e salvam em novos csv.
* tabelacao_agregado.py: Lê os novos arquivos csv, e calcula os valores agregados dos dados, agrupando-os por mes/ano e hospital. A tabela resultante é salva em "Tabela_Agregada.csv".
* tabelacao_lag_final.py: Lê a tabelacao do passo anterior e calcula as variaveis "lag", usadas no modelo para predicao. A tabela, com todos os atributos que serão usados para o modelo final, é salva em um último csv: "Tabela_lag_Final.csv".

O intuito de separar cada processo em seu próprio script é devido a robustez contra a possibilidade de erros ou mudanças, permitindo que alterações sejam feitas sem a necessidade de recomeçar o processo do zero. Além disso, cada script demora vários minutos, alguns até passando de uma hora, para terminar. Juntar todos os scripts em um resultaria em um programa que terminaria após várias horas.

### Etapa 1: Inicialização
Import's usados, e leitura dos dados 

In [8]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import sklearn as sk
import statsmodels.discrete.discrete_model as sm


df = pd.read_csv("./SIH_Dados/Tabela_lag_Final.csv")

### Etapa 2: Limpagem e separação dos dados
Retirada de linhas com dados nulos e separação dos dados de treinamento, validação e teste.

In [9]:
# TEMPORARIO ++++++++++++++++++++++++++
df.drop(columns=["J_count_ma3_lag1", "J_count_ma6_lag1", "J_count_ma12_lag1"], inplace=True)

In [60]:
df.dropna(inplace=True)

X_Train = df[(df.year >= 2022) & (df.year <= 2024)]
X_Train = X_Train[(X_Train.year < 2024) | (X_Train.month <= 6)].reset_index()
Y_Train = X_Train["J_count"]
X_Train.drop(columns="J_count", inplace=True)


X_Valid = df[(df.year == 2024) & (df.month >= 7)].reset_index()
Y_Valid = X_Valid["J_count"]
X_Valid.drop(columns="J_count", inplace=True)
X_Valid.reset_index(inplace=True)

X_Test = df[df.year == 2025].reset_index()
Y_Test = X_Test["J_count"]
X_Test.drop(columns="J_count", inplace=True)
X_Test.reset_index(inplace=True)


In [64]:
print(Y_Train)

model = sm.NegativeBinomial(Y_Train, X_Train)

res = model.fit(method='powell')

0        225
1         21
2         48
3         24
4         53
        ... 
61350      3
61351     19
61352      9
61353      4
61354      7
Name: J_count, Length: 61355, dtype: int64


/home/luis_edu/Documents/Prog/NIAR/Toy_Example/venv/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1074: RuntimeWarning: overflow encountered in exp
  return np.exp(linpred)
/home/luis_edu/Documents/Prog/NIAR/Toy_Example/venv/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1567: RuntimeWarning: overflow encountered in exp
  L = np.exp(np.dot(X,params) + exposure + offset)
/home/luis_edu/Documents/Prog/NIAR/Toy_Example/venv/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1568: RuntimeWarning: invalid value encountered in multiply
  return -np.dot(L*X.T, X)
/home/luis_edu/Documents/Prog/NIAR/Toy_Example/venv/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1568: RuntimeWarning: invalid value encountered in dot
  return -np.dot(L*X.T, X)
/home/luis_edu/Documents/Prog/NIAR/Toy_Example/venv/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1478: RuntimeWarning: overflow encountered in exp
  L =

         Current function value: nan
         Iterations: 1
         Function evaluations: 274


/home/luis_edu/Documents/Prog/NIAR/Toy_Example/venv/lib/python3.12/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/home/luis_edu/Documents/Prog/NIAR/Toy_Example/venv/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [67]:
print(res.params)


val_resul = model.predict(params=res.params, exog=X_Valid, which="prob")
test_resul = model.predict(params=res.params, exog=X_Test, which="prob")



print(val_resul.summary())
print(test_resul.summary())

index                                 NaN
year                                  NaN
month                                 NaN
CNES                                  NaN
quarter                               NaN
                                       ..
catchment_outside_local_share_lag12   NaN
J_missing_diag_sec_share_lag12        NaN
J_missing_proc_share_lag12            NaN
J_growth_1m                           NaN
alpha                                 NaN
Length: 91, dtype: float64


KeyError: -1